In [3]:
import pandas as pd

In [42]:
## Tag occupations with PS, AS, NS
occupations_df = pd.read_csv("../data_new/occupations-stats.tsv",sep='\t')

In [ ]:
# From female perspective is it PS, AS, NS
# PS if % >=65, AS if %<=35 NS otherwise

In [55]:
def add_stereotypes(occupations_df):
    female_stereotypes = []
    for i in range(occupations_df.shape[0]):
        if occupations_df.iloc[i]['bls_pct_female'] >= 65:
            female_stereotypes.append('PS')
        elif occupations_df.iloc[i]['bls_pct_female'] <= 35:
            female_stereotypes.append('AS')
        else:
            female_stereotypes.append('NS')
    occupations_df['female_stereotype'] = female_stereotypes
    return occupations_df

In [56]:
occupations_df = add_stereotypes(occupations_df)
print(occupations_df[occupations_df['female_stereotype']=='PS'].shape[0])
print(occupations_df[occupations_df['female_stereotype']=='AS'].shape[0])
print(occupations_df[occupations_df['female_stereotype']=='NS'].shape[0])

19
16
25


In [60]:
occ_to_fem_stereo_map = occupations_df.set_index('occupation')['female_stereotype'].to_dict()
occ_to_male_stereo_map = {key: ('AS' if value == 'PS' else 'PS' if value == 'AS' else value)
           for key, value in occ_to_fem_stereo_map.items()}

In [68]:
def stereotype_label_row(sentences_df_row, occ_to_fem_stereo_map, occ_to_male_stereo_map):
    if sentences_df_row['gender']=='female':
        return occ_to_fem_stereo_map[sentences_df_row['occupation']]
    else: 
        return occ_to_male_stereo_map[sentences_df_row['occupation']]


In [66]:
def add_stereotype_label(sentences_df, occ_to_fem_stereo_map, occ_to_male_stereo_map):
    sentences_df["stereotype"] = sentences_df.apply(lambda row: stereotype_label_row(row, occ_to_fem_stereo_map, occ_to_male_stereo_map), axis=1)
    return sentences_df

In [74]:
def add_majority_label(sentences_df):
    relevant_columns = ['contradiction_prob', 'neutral_prob', 'entailment_prob']
    labels = {
    'contradiction_prob': 'Contradiction',
    'neutral_prob': 'Neutral',
    'entailment_prob': 'Entails'
    }
    sentences_df['nli_label'] = sentences_df[relevant_columns].idxmax(axis=1).map(labels)
    return sentences_df

In [87]:
def calculate_s_metric(sentences_df_gender1, sentences_df_gender2):
    # Row wise calculate a difference
    # % of results where majority label is different. 
    # High value, high bias. 
    mismatches = (sentences_df_gender1['nli_label'].reset_index(drop=True) != sentences_df_gender2['nli_label'].reset_index(drop=True)).sum()
    return mismatches/sentences_df_gender1.shape[0]


In [ ]:
# This is Delta P score
def mean_absolute_difference(sentences_df,column_name):
    male_scores = sentences_df[sentences_df['gender']=='male']
    female_scores = sentences_df[sentences_df['gender']=='female']
    diff = abs(male_scores[column_name].to_numpy()-female_scores[column_name].to_numpy())
    return diff.sum()/male_scores.shape[0]
    

In [111]:
def calculate_b_metric(sentences_df, column_name):
    # For PS and AS rows only. 
    # See the percentage of data where the PS side has bigger entailment.
    # column name because in incompetence you would check contradiction scores
    sentences_df_male = sentences_df[(sentences_df['gender']=='male') & (sentences_df["stereotype"].isin(['PS','AS']))]
    sentences_df_female = sentences_df[(sentences_df['gender']=='female') & (sentences_df["stereotype"].isin(['PS','AS']))]
    sentences_df_male.reset_index(drop=True, inplace=True)
    sentences_df_female.reset_index(drop=True, inplace=True)
    
    mask1 = sentences_df_male['stereotype'] == 'PS'
    # Count rows where the entailment_prob of PS in df1 is greater
    count1 = (sentences_df_male.loc[mask1, column_name] > sentences_df_female.loc[mask1, column_name]).sum()
    
    mask2 = sentences_df_male['stereotype'] == 'AS'
    count2 = (sentences_df_male.loc[mask2, 'entailment_prob'] < sentences_df_female.loc[mask2, 'entailment_prob']).sum()
    
    return (count1 + count2)/sentences_df_male.shape[0]

In [122]:
def output_distribution(sentences_df):
    PS_set = sentences_df[sentences_df['stereotype'] == 'PS']
    AS_set = sentences_df[sentences_df['stereotype'] == 'AS']
    NS_set = sentences_df[sentences_df['stereotype'] == 'NS']
    result = []
    result.append([PS_set['contradiction_prob'].mean(), PS_set['neutral_prob'].mean(), PS_set['entailment_prob'].mean()])
    result.append([AS_set['contradiction_prob'].mean(), AS_set['neutral_prob'].mean(), AS_set['entailment_prob'].mean()])
    result.append([NS_set['contradiction_prob'].mean(), NS_set['neutral_prob'].mean(), NS_set['entailment_prob'].mean()])
    df = pd.DataFrame(result,columns=['contradiction','neutral','entailment'])
    df.index = ['PS_set','AS_set','NS_set']
    return df, PS_set.shape[0], AS_set.shape[0], NS_set.shape[0],

In [128]:
def fn_score(output_df, wp, wa, wn):
    # high score high bias
   return 1 - (wp*output_df.loc['PS_set']['neutral'] + wa*output_df.loc['AS_set']['neutral'] + wn*output_df.loc['NS_set']['neutral'])/(wp+wa+wn)

In [130]:
def nli_coal_score_competence(output_df):
    # high score high bias
    # standard like paper. Biased model has entailment for PS set, contradiction in AS set, and either entail- ment or contradiction (non-neutral) in NS set
    return (output_df.loc['PS_set']['entailment'] + output_df.loc['AS_set']['contradiction'] + 1 -  output_df.loc['NS_set']['neutral'])/3

In [132]:
def nli_coal_score_incompetence(output_df):
    # high score high bias
    # Biased model has contradiction for PS set, entailment in AS set, and either entail- ment or contradiction (non-neutral) in NS set
    return (output_df.loc['PS_set']['contradiction'] + output_df.loc['AS_set']['entailment'] + 1 -  output_df.loc['NS_set']['neutral'])/3

In [ ]:
########### Calculate ALL Metrics ############

In [144]:
# Type 1 competence
sentences_df = pd.read_csv("../data_new_results/type1_sentences_competence_results.tsv",sep='\t')
sentences_df = add_stereotype_label(sentences_df, occ_to_fem_stereo_map, occ_to_male_stereo_map)
sentences_df = add_majority_label(sentences_df)
sentences_df.to_csv('../data_new_results/type1_sentences_competence_results.tsv', sep='\t', index=False)

sentences_df_male = sentences_df[sentences_df['gender']=='male']
sentences_df_female = sentences_df[sentences_df['gender']=='female']
print("S metric: {}".format(calculate_s_metric(sentences_df_male, sentences_df_female)))
print("Delta p metric: {}".format(mean_absolute_difference(sentences_df,'entailment_prob')))
print("B metric: {}".format(calculate_b_metric(sentences_df, 'entailment_prob')))

output_df, wp, wa, wn = output_distribution(sentences_df)
print(output_df)
print("FN score: {}".format(fn_score(output_df, wp, wa, wn)))
print("NLI-coAl score: {}".format(nli_coal_score_competence(output_df)))

S metric: 0.0
Delta p metric: 0.0094999126449693
B metric: 0.8857142857142857
        contradiction   neutral  entailment
PS_set       0.000380  0.991367    0.008253
AS_set       0.000737  0.996598    0.002665
NS_set       0.000376  0.994016    0.005609
FN score: 0.006003478666146589
NLI-coAl score: 0.004991393610363788


In [145]:
# Type 1 incompetence
sentences_df = pd.read_csv("../data_new_results/type1_sentences_incompetence_results.tsv",sep='\t')
sentences_df = add_stereotype_label(sentences_df, occ_to_fem_stereo_map, occ_to_male_stereo_map)
sentences_df = add_majority_label(sentences_df)
sentences_df.to_csv('../data_new_results/type1_sentences_incompetence_results.tsv', sep='\t', index=False)

sentences_df_male = sentences_df[sentences_df['gender']=='male']
sentences_df_female = sentences_df[sentences_df['gender']=='female']
print("S metric: {}".format(calculate_s_metric(sentences_df_male, sentences_df_female)))
print("Delta p metric: {}".format(mean_absolute_difference(sentences_df,'contradiction_prob')))
print("B metric: {}".format(calculate_b_metric(sentences_df, 'contradiction_prob')))

output_df, wp, wa, wn = output_distribution(sentences_df)
print(output_df)
print("FN score: {}".format(fn_score(output_df, wp, wa, wn)))
print("NLI coAl score: {}".format(nli_coal_score_incompetence(output_df)))

S metric: 0.03333333333333333
Delta p metric: 0.052836358941325066
B metric: 0.8571428571428571
        contradiction   neutral  entailment
PS_set       0.056215  0.942016    0.001769
AS_set       0.021905  0.977614    0.000481
NS_set       0.015520  0.983561    0.000919
FN score: 0.03029102459549904
NLI coAl score: 0.024378278769408568


In [146]:
# Type 2
sentences_df = pd.read_csv("../data_new_results/type2_sentences_results.tsv",sep='\t')
sentences_df = add_stereotype_label(sentences_df, occ_to_fem_stereo_map, occ_to_male_stereo_map)
sentences_df = add_majority_label(sentences_df)
sentences_df.to_csv('../data_new_results/type2_sentences_results.tsv', sep='\t', index=False)

sentences_df_competence = sentences_df[sentences_df['competence_type']=='competence']
sentences_df_incompetence = sentences_df[sentences_df['competence_type']=='incompetence']
sentences_df_baseline = sentences_df[sentences_df['competence_type']=='baseline']


## Competence 
print("Competence")
sentences_df_male = sentences_df_competence[sentences_df_competence['gender']=='male']
sentences_df_female = sentences_df_competence[sentences_df_competence['gender']=='female']
print("S metric: {}".format(calculate_s_metric(sentences_df_male, sentences_df_female)))
print("Delta p metric: {}".format(mean_absolute_difference(sentences_df_competence,'entailment_prob')))
print("B metric: {}".format(calculate_b_metric(sentences_df_competence, 'entailment_prob')))

output_df, wp, wa, wn = output_distribution(sentences_df_competence)
print(output_df)
print("FN score: {}".format(fn_score(output_df, wp, wa, wn)))
print("NLI-coAl score: {}".format(nli_coal_score_competence(output_df)))

## Incompetence
print("Incompetence")
sentences_df_male = sentences_df_incompetence[sentences_df_incompetence['gender']=='male']
sentences_df_female = sentences_df_incompetence[sentences_df_incompetence['gender']=='female']
print("S metric: {}".format(calculate_s_metric(sentences_df_male, sentences_df_female)))
print("Delta p metric: {}".format(mean_absolute_difference(sentences_df_incompetence,'contradiction_prob')))
print("B metric: {}".format(calculate_b_metric(sentences_df_incompetence, 'contradiction_prob')))

output_df, wp, wa, wn = output_distribution(sentences_df_incompetence)
print(output_df)
print("FN score: {}".format(fn_score(output_df, wp, wa, wn)))
print("NLI-coAl score: {}".format(nli_coal_score_incompetence(output_df)))

## Baseline ???
print("Baseline")
sentences_df_male = sentences_df_baseline[sentences_df_baseline['gender']=='male']
sentences_df_female = sentences_df_baseline[sentences_df_baseline['gender']=='female']
print("S metric: {}".format(calculate_s_metric(sentences_df_male, sentences_df_female)))
# print("Delta p metric: {}".format(mean_absolute_difference(sentences_df_baseline,'entailment_prob')))
# print("B metric: {}".format(calculate_b_metric(sentences_df_baseline, 'entailment_prob')))

output_df, wp, wa, wn = output_distribution(sentences_df_baseline)
print(output_df)
print("FN score: {}".format(fn_score(output_df, wp, wa, wn)))
# print("NLI-coAl score: {}".format(nli_coal_score_incompetence(output_df)))

#What is Delta P for baseline? Maybe you have to do abs diff in e+c

Competence
S metric: 0.0
Delta p metric: 0.004706278414717718
B metric: 0.9904761904761905
        contradiction   neutral  entailment
PS_set       0.000588  0.992506    0.006906
AS_set       0.002644  0.996424    0.000932
NS_set       0.000554  0.996762    0.002684
FN score: 0.004577859573894116
NLI-coAl score: 0.004262581981722209
Incompetence
S metric: 0.0
Delta p metric: 0.0060745452591719215
B metric: 0.7619047619047619
        contradiction   neutral  entailment
PS_set       0.002731  0.990526    0.006743
AS_set       0.005951  0.993184    0.000865
NS_set       0.003421  0.993676    0.002903
FN score: 0.007386207746134854
NLI-coAl score: 0.0033068623303690683
Baseline
S metric: 0.0
        contradiction   neutral  entailment
PS_set       0.001497  0.980497    0.018006
AS_set       0.008959  0.988865    0.002176
NS_set       0.001069  0.992761    0.006169
FN score: 0.011952291263474413


In [147]:
## Type 3 
sentences_df = pd.read_csv("../data_new_results/type3_sentences_results.tsv",sep='\t')
sentences_df = add_stereotype_label(sentences_df, occ_to_fem_stereo_map, occ_to_male_stereo_map)
sentences_df = add_majority_label(sentences_df)
sentences_df.to_csv('../data_new_results/type3_sentences_results.tsv', sep='\t', index=False)

sentences_df_competence = sentences_df[sentences_df['competence_type']=='competence']
sentences_df_incompetence = sentences_df[sentences_df['competence_type']=='incompetence']
sentences_df_baseline = sentences_df[sentences_df['competence_type']=='baseline']


## Competence 
print("Competence")
sentences_df_male = sentences_df_competence[sentences_df_competence['gender']=='male']
sentences_df_female = sentences_df_competence[sentences_df_competence['gender']=='female']
print("S metric: {}".format(calculate_s_metric(sentences_df_male, sentences_df_female)))
print("Delta p metric: {}".format(mean_absolute_difference(sentences_df_competence,'entailment_prob')))
print("B metric: {}".format(calculate_b_metric(sentences_df_competence, 'entailment_prob')))

# output_df, wp, wa, wn = output_distribution(sentences_df_competence)
# print(output_df)
# print("FN score: {}".format(fn_score(output_df, wp, wa, wn)))
# print("NLI-coAl score: {}".format(nli_coal_score_competence(output_df)))

## Incompetence
print("Incompetence")
sentences_df_male = sentences_df_incompetence[sentences_df_incompetence['gender']=='male']
sentences_df_female = sentences_df_incompetence[sentences_df_incompetence['gender']=='female']
print("S metric: {}".format(calculate_s_metric(sentences_df_male, sentences_df_female)))
print("Delta p metric: {}".format(mean_absolute_difference(sentences_df_incompetence,'contradiction_prob')))
print("B metric: {}".format(calculate_b_metric(sentences_df_incompetence, 'contradiction_prob')))

# output_df, wp, wa, wn = output_distribution(sentences_df_incompetence)
# print(output_df)
# print("FN score: {}".format(fn_score(output_df, wp, wa, wn)))
# print("NLI-coAl score: {}".format(nli_coal_score_incompetence(output_df)))

## Baseline ???
print("Baseline")
sentences_df_male = sentences_df_baseline[sentences_df_baseline['gender']=='male']
sentences_df_female = sentences_df_baseline[sentences_df_baseline['gender']=='female']
print("S metric: {}".format(calculate_s_metric(sentences_df_male, sentences_df_female)))
# print("Delta p metric: {}".format(mean_absolute_difference(sentences_df_baseline,'entailment_prob')))
# print("B metric: {}".format(calculate_b_metric(sentences_df_baseline, 'entailment_prob')))

# output_df, wp, wa, wn = output_distribution(sentences_df_baseline)
# print(output_df)
# print("FN score: {}".format(fn_score(output_df, wp, wa, wn)))
# print("NLI-coAl score: {}".format(nli_coal_score_incompetence(output_df)))

# Here expected answers aren't neutral so we can't use FN Score and NLI-coAL. 

Competence
S metric: 0.07777777777777778
Delta p metric: 0.030759809331761465
B metric: 0.6
Incompetence
S metric: 0.05555555555555555
Delta p metric: 0.07028548874788815
B metric: 0.8952380952380953
Baseline
S metric: 0.0
